# ***<span style="color: yellow; background-color: black; border-radius: 10px; padding: 10px;">CNN - Convolutional Neural Networks</span>***

In [54]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### ***Definindo parãmetros***

In [55]:
# Definir o tamanho do vetor de ruído de entrada para o gerador
epochs = 1000
batch_size = 32
image_size = (150, 150)
NUM_CLASSES = 10

### ***Carregando e pré processando***

In [56]:
dataset_dir = 'dataset'
rotulos = sorted(os.listdir(dataset_dir))
classe_para_rotulo = {indice: rotulo for indice, rotulo in enumerate(rotulos)}
print(classe_para_rotulo)

# Use o ImageDataGenerator para carregar e pré-processar as imagens
data_generator = ImageDataGenerator(rescale=1.0/255.0)

# Crie um gerador de fluxo de dados para treinamento
train_generator = data_generator.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)


{0: 'cane', 1: 'cavallo', 2: 'elefante', 3: 'farfalla', 4: 'gallina', 5: 'gatto', 6: 'mucca', 7: 'pecora', 8: 'ragno', 9: 'scoiattolo'}
Found 26179 images belonging to 10 classes.


### ***Criando modelo***

In [57]:
def create_model():
    model = keras.Sequential([
        keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(150, 150, 3)),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),

        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        
        keras.layers.Dense(units=512, activation='relu'),
        keras.layers.Dense(units=NUM_CLASSES, activation='softmax')
    ])
    
    return model


### ***Compilando o modelo***

In [58]:
model = create_model()
# Compile o modelo
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

### ***Treinando o modelo***

In [59]:

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

model.fit(x=train_generator, epochs=epochs, callbacks=[callbacks])

Epoch 1/1000
819/819 [==============================] - 982s 1s/step - loss: 1.7621 - accuracy: 0.3837
Epoch 2/1000
592/819 [====================>.........] - ETA: 4:07 - loss: 1.2878 - accuracy: 0.5582

### ***Salvando modelo já treinado***

In [ ]:
model.save("model_cnn.h5")

### ***Carregando o modelo para testes***

In [ ]:
# Carregue o modelo treinado
modelo_carregado = load_model("model_cnn.h5")

### ***Fazendo predições***

In [ ]:

# Carregue a imagem que você deseja prever
imagem = image.load_img(input("Digite o caminho da imagem >>> "), target_size=image_size)

# Converta a imagem em um array numpy
imagem_array = image.img_to_array(imagem)

# Expanda as dimensões para corresponder ao formato de entrada do modelo
imagem_array = np.expand_dims(imagem_array, axis=0)

# Normalize os valores dos pixels (escala de 0 a 1)
imagem_array /= 255.0

# Faça a previsão
previsao = modelo_carregado.predict(imagem_array)

# Determine a classe com a probabilidade mais alta
classe_predita = np.argmax(previsao, axis=1)

# Crie a mensagem de saída com base na classe predita
mensagem = f"A imagem que você enviou foi classificada como {classe_para_rotulo[classe_predita[0]]}."

In [ ]:
mensagem